In [23]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [13]:
import pandas as pd
import numpy as np
import math

## Valle rmse

In [3]:
#FILE NAME RENAMED WRONG, IT IS ACTUALLY ASV!!!!
valle_asv = pd.read_csv('../data/output/Valle_ALS_validaatiopisteet_vertailuun_siivottu_2.csv', delimiter=';')

In [4]:
valle_asv

,PointNo,Elevation_orthometricH,Latitude,Longitude,Height_ellipsoidalH,ASV_depth,Geoid_h,Water_lvl,ASV_ElevationH,Validation_depth,ASV_Height_ellipsoidalH
0,valle14,64.832,69.925991,27.001370,89.137092,-1.168000,24.52,64.41,63.242000,-1.590000,87.762000
1,valle12,64.710,69.926008,27.001523,89.015000,-1.238200,24.52,64.41,63.171800,-1.538200,87.691800
2,valle26,64.875,69.925940,27.001467,89.180286,-0.908000,24.52,64.41,63.502000,-1.373000,88.022000
3,valle17,65.037,69.926075,27.001217,89.342939,-0.082000,24.52,64.41,64.328000,-0.709000,88.848000
4,valle15,64.981,69.925987,27.001334,89.286498,-1.155102,24.52,64.41,63.254898,-1.726102,87.774898


In [6]:
valle_als = pd.read_csv('../data/output/Valle_ALS_validaatiopisteet_vertailuun_siivottu.csv', delimiter=';')

In [7]:
valle_als

,Latitude,Longitude,Height,IDW
0,69.925891,27.001095,90.322226,89.922427
1,69.925836,27.001092,90.577668,91.074363
2,69.925814,27.001266,90.286394,90.332491
3,69.925784,27.001441,89.975550,90.559079
4,69.925791,27.001553,89.580965,87.827659
5,69.925809,27.001623,89.449445,87.905665
6,69.925835,27.001534,89.612676,89.634905
7,69.925848,27.001642,89.241923,87.853051
8,69.925882,27.001693,89.112957,89.101316
9,69.925926,27.001660,89.041733,89.003451


In [15]:
asv_actual = valle_asv['ASV_depth'].values.tolist()

In [16]:
asv_actual

[-1.1680001, -1.2382, -0.90799995, -0.081999984, -1.1551018]

In [17]:
asv_predicted = valle_asv['Validation_depth'].values.tolist()

In [18]:
asv_predicted

[-1.5900001, -1.5382, -1.37299995, -0.708999984, -1.7261018]

In [19]:
MSE = mean_squared_error(asv_actual, asv_predicted)
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:\n")
print(RMSE)

Root Mean Square Error:

0.49060758249338143


In [20]:
als_actual = valle_als['Height'].values.tolist()

In [21]:
als_predicted = valle_als['IDW'].values.tolist()

In [22]:
MSE = mean_squared_error(als_actual, als_predicted)
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:\n")
print(RMSE)

Root Mean Square Error:

0.8131865198308067


In [24]:
MAE = mean_absolute_error(als_actual, als_predicted)

print(MAE)

0.5743318585227102


In [25]:
MAE = mean_absolute_error(asv_actual, asv_predicted)

print(MAE)

0.477
